In [45]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
tqdm.pandas()

import pickle
import pyarrow.parquet as pq

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [141]:
import ipyparallel
# ipcluster nbextension enable --user

In [2]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'

In [44]:
FTRS = ['last_target'# last known target for this user (by last session)
        ,'last_moment_invest'
        ,'last_moment_mobile_recharge'
        ,'last_moment_statement'
        ,'last_moment_chat'
        ,'last_moment_card2card_transfer'
        ,'last_moment_phone_money_transfer'
        ,'last_moment_card_recharge'
        ,'last_moment_main_screen'
        ,'last_moment_own_transfer'
        ,'last_moment_credit_info'
        
         #most frequent in this 
        ,'mf_dow'    # day of week
        ,'mf_dom'    # day of month
        ,'mf_pod'    # part of day
        
        #most frequent for this user in this 
         'mf_this_user_dow'    # day of week
        ,'mf_this_user_dom'    # day of month
        ,'mf_this_user_pod'    # part of day
        
        # days from last goal was a 
        ,'days_since_last_invest'        # invest
        ,'days_since_last_statement'     # statement
        ,'days_since_last_chat'          # chat
        ,'days_since_last_card_recharge' # card_recharge
        ,'days_since_last_main_screen'   # main_screen
        ,'days_since_last_own_transfer'  # own_transfer
        ,'days_since_last_credit_info'   # credit_info
        ,'days_since_last_mobile_recharge'      # mobile_recharge
        ,'days_since_last_card2card_transfer'   # card2card_transfer
        ,'days_since_last_phone_money_transfer' # phone_money_transfer
    
        
        #, last several days
       ]

In [30]:
possible_targets = ['invest', 'mobile_recharge', 'statement', 'chat', 'card2card_transfer', 
                    'phone_money_transfer', 'card_recharge', 'main_screen', 'own_transfer', 
                    'credit_info']

In [5]:
files = sorted(glob(os.path.join(DATA, CLICKSTREAM, 'part*')))
#files

In [ ]:
sample = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_sample_prediction.csv'))
test   = pd.read_csv(os.path.join(DATA, 'alfabattle2_prediction_session_timestamp.csv'))

In [58]:
targets_df = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp'])
targets_df.sort_values(by='timestamp', ascending = True, inplace = True)

In [7]:
# session wich has a target
sess_with_target = list(targets_df.session_id.values)
uniq_client_pin = list(targets_df.client_pin.unique())

In [46]:
summarize_df = pd.DataFrame(index = range(len(uniq_client_pin)), columns = FTRS)

In [47]:
summarize_df['client_pin'] = uniq_client_pin

In [137]:
data = pd.read_parquet(files[0])
data.sort_values(by='timestamp', ascending=True, inplace = True)
data.shape

(12468595, 15)

In [138]:
len(data.client.unique())

8211

In [139]:
timestamp = pd.Timestamp('2000-01-01')

for client in tqdm(data.client.unique()):
    index = int(summarize_df[summarize_df.client_pin == client].index[0])
    #print(index)
    for targ in possible_targets:
        tmp_list = list(targets_df.query('client_pin == @client & multi_class_target == @targ').timestamp)
        if len(tmp_list) == 0:
            hrs_mean = 0.0
            hrs_std = 0.0
            last = timestamp
        else:
            diff_list = [0]*(len(tmp_list) - 1)
            for idx in range(1, len(tmp_list)):
                diff_list[idx - 1] = (tmp_list[idx] - tmp_list[idx-1]).total_seconds() // 3600

            hrs_mean = np.mean(diff_list)
            hrs_std = np.std(diff_list)
            last = tmp_list[-1]
            
        summarize_df.loc[index, 'hrs_between_' + targ + '_mean'] = hrs_mean
        summarize_df.loc[index, 'hrs_between_' + targ + '_std'] = hrs_std
        summarize_df.loc[index, 'last_moment_' + targ] = last
        
    
            
print('done')    

  5%|███▉                                                                         | 414/8211 [28:47<9:02:23,  4.17s/it]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [97]:
summarize_df[summarize_df.client_pin == data.client.unique()[0]]

,last_target,last_moment_invest,last_moment_mobile_recharge,last_moment_statement,last_moment_chat,last_moment_card2card_transfer,last_moment_phone_money_transfer,last_moment_card_recharge,last_moment_main_screen,last_moment_own_transfer,...,hrs_between_phone_money_transfer_mean,hrs_between_phone_money_transfer_std,hrs_between_card_recharge_mean,hrs_between_card_recharge_std,hrs_between_main_screen_mean,hrs_between_main_screen_std,hrs_between_own_transfer_mean,hrs_between_own_transfer_std,hrs_between_credit_info_mean,hrs_between_credit_info_std
20690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1203.0,1105.0,212.64,276.855324,183.551724,201.278772,1899.0,0.0,0.0,0.0


In [ ]:
# calculate one in second

In [17]:
%%time
# collect statistic about every client
cnt = 0
everyday = []
for client in tqdm(data.client.unique()[:50]):
    # for each session
    current = ''
    weekend_start_hour = []
    weekday_start_hour = []
    everyday_cnt = []
    for idx in data[data.client == client].index:
        if  (data.loc[idx, 'session_id'] != current):# and (data.loc[idx, 'session_id'] in sess_with_target):
            cnt+= 1
            current = data.loc[idx, 'session_id']
            if data.loc[idx, 'timestamp'].weekday() >= 5:
                weekend_start_hour.append(data.loc[idx, 'timestamp'].hour)
            else:
                weekday_start_hour.append(data.loc[idx, 'timestamp'].hour)
            
            everyday_cnt.append(data.loc[idx, 'timestamp'].weekday())
            
    everyday.append(everyday_cnt)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:37<00:00,  1.32it/s]

Wall time: 39 s


In [ ]:
print(cnt)

In [ ]:
len(data.session_id.unique())

In [ ]:
pred_sess = ''
dow = []
dom = [] 
weekend_start_hour = []
weekday_start_hour = []

#for idx, el in enumerate(data.session_id):
#for idx in tqdm(range(data.shape[0])):
for idx in tqdm(range(5000)):
    #if (el != pred_sess) and (el in sess_with_target):
    if ( data.loc[idx, 'session_id'] != pred_sess) and (data.loc[idx, 'session_id'] in sess_with_target):
        pred_sess = data.loc[idx, 'session_id']
        if (data.loc[idx, 'timestamp'].weekday() >= 5):
            weekend_start_hour    
        #get start time
        #start_hour.append(data.loc[idx, 'timestamp'].hour)
        #get start day of week
        #dow.append(data.loc[idx, 'timestamp'].weekday())
        #get start day of month
        #dom.append(data.loc[idx, 'timestamp'].day)
        #print(el)
        break

In [ ]:
data.loc[0, 'timestamp'].weekday()

In [ ]:
data_target[data_target.session_id == data.loc[0, 'session_id']]

In [ ]:
data_target.keys()

In [ ]:
data.loc[1920, 'session_id'] in sess_with_target

In [ ]:
data.loc[1920, 'session_id']

In [ ]:
%%time
data_target['dow'] = data_target.timestamp.map(lambda x: x.weekday())
data_target['dom'] = data_target.timestamp.map(lambda x: x.day)
data_target['start_hour'] = data_target.timestamp.map(lambda x: x.hour)

In [ ]:
data_target.head()

In [ ]:
data_target.groupby('dow').multi_class_target.agg(pd.Series.mode)

In [ ]:
realy = data_target.groupby('dow').multi_class_target.value_counts()

In [ ]:
realy

In [ ]:
data_target.groupby('dom').multi_class_target.agg(pd.Series.mode)

In [ ]:
data_target.groupby('start_hour').multi_class_target.value_counts()

In [ ]:
data_target.multi_class_target.value_counts().index[1]